In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import re
import shutil

# Refer to requirements.txt, use a venv

# Find data folder path and csv files stored in list

In [22]:
data_path = os.path.join('..', 'data')
csv_files = sorted(glob.glob(os.path.join(data_path, '*.csv')))

print(f'Found {len(csv_files)} files') # Found 12 files
print(csv_files)
print(list(map(lambda file: os.path.basename(file), csv_files)))

Found 12 files
['..\\data\\0.5m.1.csv', '..\\data\\0.5m.2.csv', '..\\data\\0.5m.3.csv', '..\\data\\1.5m.1.csv', '..\\data\\1.5m.2.csv', '..\\data\\1.5m.3.csv', '..\\data\\1m.1.csv', '..\\data\\1m.2.csv', '..\\data\\1m.3.csv', '..\\data\\2m.1.csv', '..\\data\\2m.2.csv', '..\\data\\2m.3.csv']
['0.5m.1.csv', '0.5m.2.csv', '0.5m.3.csv', '1.5m.1.csv', '1.5m.2.csv', '1.5m.3.csv', '1m.1.csv', '1m.2.csv', '1m.3.csv', '2m.1.csv', '2m.2.csv', '2m.3.csv']


In [ ]:
plot_dir = '../plots'
os.makedirs(plot_dir, exist_ok=True)

trials = []
pattern = r"([0-9.]+)?m\.([0-9]+)\.csv"

if os.path.exists(plot_dir):
    shutil.rmtree(plot_dir)
os.makedirs(plot_dir)

for file in csv_files:
    base = os.path.basename(file)
    cur_df = pd.read_csv(file)
    match = re.search(pattern, base)
    
    if match:
        height = float(match.group(1))
        trial = int(match.group(2))
        
        cur_df['Height (m)'] = height
        cur_df['Trial'] = trial
        
        impact_idx = cur_df['atotal'].idxmax()
        plot_df = cur_df.iloc[:impact_idx + 15].copy()
        
        plt.figure(figsize=(8, 4))
        plt.plot(plot_df['time'], plot_df['ax'], label='$a_x$')
        plt.plot(plot_df['time'], plot_df['ay'], label='$a_y$')
        plt.plot(plot_df['time'], plot_df['az'], label='$a_z$')
        
        plt.title(f"Acceleration Profile: {height}m - Trial {trial}")
        plt.xlabel("Time (s)")
        plt.ylabel("Acceleration ($m/s^2$)")
        plt.ylim((-25,25))
        plt.legend()    
        plt.grid(True, alpha=0.3)
        
        plot_filename = f"plot_{height}m_trial_{trial}_axis.png"
        plt.savefig(os.path.join(plot_dir, plot_filename))
        plt.close()

        plt.figure(figsize=(8, 4))
        plt.plot(plot_df['time'], plot_df['atotal'], color='black', label='$a_{total}$', linewidth=1, alpha=0.7)
        plt.axhline(y=9.8, color='red', label='g=9.8m/s^2', linestyle='dotted')
        
        plt.title(f"Acceleration Profile: {height}m - Trial {trial}")
        plt.xlabel("Time (s)")
        plt.ylabel("Acceleration ($m/s^2$)")
        plt.ylim((0,25))
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plot_filename = f"plot_{height}m_trial_{trial}_total.png"
        plt.savefig(os.path.join(plot_dir, plot_filename))
        plt.close()
        
        trials.append(cur_df)